https://www.kaggle.com/nvpsani/topic-modelling-using-guided-lda

https://medium.com/analytics-vidhya/how-i-tackled-a-real-world-problem-with-guidedlda-55ee803a6f0d

In [13]:
!pip install lda

     |████████████████████████████████| 344 kB 3.0 MB/s 
     |████████████████████████████████| 99 kB 11.0 MB/s 
  Attempting uninstall: pbr
    Found existing installation: pbr 5.5.1
    Uninstalling pbr-5.5.1:
      Successfully uninstalled pbr-5.5.1


# GuidedLDA utilities

In [17]:
from __future__ import absolute_import, unicode_literals  # noqa

import logging
import numbers
import sys

import numpy as np

PY2 = sys.version_info[0] == 2
if PY2:
    import itertools
    zip = itertools.izip


# logger = logging.getLogger('guidedlda')
## edit 1.9.2020
logger = logging.getLogger('lda')


def check_random_state(seed):
    if seed is None:
        # i.e., use existing RandomState
        return np.random.mtrand._rand
    if isinstance(seed, (numbers.Integral, np.integer)):
        return np.random.RandomState(seed)
    if isinstance(seed, np.random.RandomState):
        return seed
    raise ValueError("{} cannot be used as a random seed.".format(seed))


def matrix_to_lists(doc_word):
    """Convert a (sparse) matrix of counts into arrays of word and doc indices
    Parameters
    ----------
    doc_word : array or sparse matrix (D, V)
        document-term matrix of counts
    Returns
    -------
    (WS, DS) : tuple of two arrays
        WS[k] contains the kth word in the corpus
        DS[k] contains the document index for the kth word
    """
    if np.count_nonzero(doc_word.sum(axis=1)) != doc_word.shape[0]:
        logger.warning("all zero row in document-term matrix found")
    if np.count_nonzero(doc_word.sum(axis=0)) != doc_word.shape[1]:
        logger.warning("all zero column in document-term matrix found")
    sparse = True
    try:
        # if doc_word is a scipy sparse matrix
        doc_word = doc_word.copy().tolil()
    except AttributeError:
        sparse = False

    if sparse and not np.issubdtype(doc_word.dtype, int):
        raise ValueError("expected sparse matrix with integer values, found float values")

    ii, jj = np.nonzero(doc_word)
    if sparse:
        ss = tuple(doc_word[i, j] for i, j in zip(ii, jj))
    else:
        ss = doc_word[ii, jj]

    n_tokens = int(doc_word.sum())
    DS = np.repeat(ii, ss).astype(np.intc)
    WS = np.empty(n_tokens, dtype=np.intc)
    startidx = 0
    for i, cnt in enumerate(ss):
        cnt = int(cnt)
        WS[startidx:startidx + cnt] = jj[i]
        startidx += cnt
    return WS, DS


def lists_to_matrix(WS, DS):
    """Convert array of word (or topic) and document indices to doc-term array
    Parameters
    -----------
    (WS, DS) : tuple of two arrays
        WS[k] contains the kth word in the corpus
        DS[k] contains the document index for the kth word
    Returns
    -------
    doc_word : array (D, V)
        document-term array of counts
    """
    D = max(DS) + 1
    V = max(WS) + 1
    doc_word = np.empty((D, V), dtype=np.intc)
    for d in range(D):
        for v in range(V):
            doc_word[d, v] = np.count_nonzero(WS[DS == d] == v)
    return doc_word


def dtm2ldac(dtm, offset=0):
    """Convert a document-term matrix into an LDA-C formatted file
    Parameters
    ----------
    dtm : array of shape N,V
    Returns
    -------
    doclines : iterable of LDA-C lines suitable for writing to file
    Notes
    -----
    If a format similar to SVMLight is desired, `offset` of 1 may be used.
    """
    try:
        dtm = dtm.tocsr()
    except AttributeError:
        pass
    assert np.issubdtype(dtm.dtype, int)
    n_rows = dtm.shape[0]
    for i, row in enumerate(dtm):
        try:
            row = row.toarray().squeeze()
        except AttributeError:
            pass
        unique_terms = np.count_nonzero(row)
        if unique_terms == 0:
            raise ValueError("dtm row {} has all zero entries.".format(i))
        term_cnt_pairs = [(i + offset, cnt) for i, cnt in enumerate(row) if cnt > 0]
        docline = str(unique_terms) + ' '
        docline += ' '.join(["{}:{}".format(i, cnt) for i, cnt in term_cnt_pairs])
        if (i + 1) % 1000 == 0:
            logger.info("dtm2ldac: on row {} of {}".format(i + 1, n_rows))
        yield docline


def ldac2dtm(stream, offset=0):
    """Convert an LDA-C formatted file to a document-term array
    Parameters
    ----------
    stream: file object
        File yielding unicode strings in LDA-C format.
    Returns
    -------
    dtm : array of shape N,V
    Notes
    -----
    If a format similar to SVMLight is the source, an `offset` of 1 may be used.
    """
    doclines = stream

    # We need to figure out the dimensions of the dtm.
    N = 0
    V = -1
    data = []
    for l in doclines:
        l = l.strip()
        # skip empty lines
        if not l:
            continue
        unique_terms = int(l.split(' ')[0])
        term_cnt_pairs = [s.split(':') for s in l.split(' ')[1:]]
        for v, _ in term_cnt_pairs:
            # check that format is indeed LDA-C with the appropriate offset
            if int(v) == 0 and offset == 1:
                raise ValueError("Indexes in LDA-C are offset 1")
        term_cnt_pairs = tuple((int(v) - offset, int(cnt)) for v, cnt in term_cnt_pairs)
        np.testing.assert_equal(unique_terms, len(term_cnt_pairs))
        V = max(V, *[v for v, cnt in term_cnt_pairs])
        data.append(term_cnt_pairs)
        N += 1
    V = V + 1
    dtm = np.zeros((N, V), dtype=np.intc)
    for i, doc in enumerate(data):
        for v, cnt in doc:
            np.testing.assert_equal(dtm[i, v], 0)
            dtm[i, v] = cnt
    return dtm

# Guided LDA

In [19]:
# coding=utf-8
"""Latent Dirichlet allocation using collapsed Gibbs sampling"""

from __future__ import absolute_import, division, unicode_literals  # noqa
import logging
import sys

import numpy as np

import lda._lda
# import lda.utils
## edit 1.9.2020

import random

logger = logging.getLogger('lda')

PY2 = sys.version_info[0] == 2
if PY2:
    range = xrange


class GuidedLDA:
    """Guided Latent Dirichlet allocation using collapsed Gibbs sampling
    Parameters
    ----------
    n_topics : int
        Number of topics
    n_iter : int, default 2000
        Number of sampling iterations
    alpha : float, default 0.1
        Dirichlet parameter for distribution over topics
    eta : float, default 0.01
        Dirichlet parameter for distribution over words
    random_state : int or RandomState, optional
        The generator used for the initial topics.
    Attributes
    ----------
    `components_` : array, shape = [n_topics, n_features]
        Point estimate of the topic-word distributions (Phi in literature)
    `topic_word_` :
        Alias for `components_`
    `word_topic_` : array, shape = [n_features, n_topics]
        Point estimate of the word-topic distributions
    `nzw_` : array, shape = [n_topics, n_features]
        Matrix of counts recording topic-word assignments in final iteration.
    `ndz_` : array, shape = [n_samples, n_topics]
        Matrix of counts recording document-topic assignments in final iteration.
    `doc_topic_` : array, shape = [n_samples, n_features]
        Point estimate of the document-topic distributions (Theta in literature)
    `nz_` : array, shape = [n_topics]
        Array of topic assignment counts in final iteration.
    Examples
    --------
    >>> import numpy
    >>> X = numpy.array([[1,1], [2, 1], [3, 1], [4, 1], [5, 8], [6, 1]])
    >>> import lda
    >>> model = lda.LDA(n_topics=2, random_state=0, n_iter=100)
    >>> model.fit(X) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    LDA(alpha=...
    >>> model.components_
    array([[ 0.85714286,  0.14285714],
           [ 0.45      ,  0.55      ]])
    >>> model.loglikelihood() #doctest: +ELLIPSIS
    -40.395...
    References
    ----------
    Blei, David M., Andrew Y. Ng, and Michael I. Jordan. "Latent Dirichlet
    Allocation." Journal of Machine Learning Research 3 (2003): 993–1022.
    Griffiths, Thomas L., and Mark Steyvers. "Finding Scientific Topics."
    Proceedings of the National Academy of Sciences 101 (2004): 5228–5235.
    doi:10.1073/pnas.0307752101.
    Wallach, Hanna, David Mimno, and Andrew McCallum. "Rethinking LDA: Why
    Priors Matter." In Advances in Neural Information Processing Systems 22,
    edited by Y.  Bengio, D. Schuurmans, J. Lafferty, C. K. I. Williams, and A.
    Culotta, 1973–1981, 2009.
    Buntine, Wray. "Estimating Likelihoods for Topic Models." In Advances in
    Machine Learning, First Asian Conference on Machine Learning (2009): 51–64.
    doi:10.1007/978-3-642-05224-8_6.
    """

    def __init__(self, n_topics, n_iter=2000, alpha=0.01, eta=0.01, random_state=None,
                 refresh=10):
        self.n_topics = n_topics
        self.n_iter = n_iter
        self.alpha = alpha
        self.eta = eta
        # if random_state is None, check_random_state(None) does nothing
        # other than return the current numpy RandomState
        self.random_state = random_state
        self.refresh = refresh

        if alpha <= 0 or eta <= 0:
            raise ValueError("alpha and eta must be greater than zero")

        # random numbers that are reused
        rng = lda.utils.check_random_state(random_state)
        if random_state:
            random.seed(random_state)
        self._rands = rng.rand(1024**2 // 8)  # 1MiB of random variates

        # configure console logging if not already configured
        if len(logger.handlers) == 1 and isinstance(logger.handlers[0], logging.NullHandler):
            logging.basicConfig(level=logging.INFO)

    def fit(self, X, y=None, seed_topics={}, seed_confidence=0):
        """Fit the model with X.
        Parameters
        ----------
        X: array-like, shape (n_samples, n_features)
            Training data, where n_samples in the number of samples
            and n_features is the number of features. Sparse matrix allowed.
        Returns
        -------
        self : object
            Returns the instance itself.
        """

        self._fit(X, seed_topics=seed_topics, seed_confidence=seed_confidence)
        return self

    def fit_transform(self, X, y=None, seed_topics={}, seed_confidence=0):
        """Apply dimensionality reduction on X
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            New data, where n_samples in the number of samples
            and n_features is the number of features. Sparse matrix allowed.
        Returns
        -------
        doc_topic : array-like, shape (n_samples, n_topics)
            Point estimate of the document-topic distributions
        """
        if isinstance(X, np.ndarray):
            # in case user passes a (non-sparse) array of shape (n_features,)
            # turn it into an array of shape (1, n_features)
            X = np.atleast_2d(X)
        self._fit(X, seed_topics=seed_topics, seed_confidence=seed_confidence)
        return self.doc_topic_

    def transform(self, X, max_iter=20, tol=1e-16):
        """Transform the data X according to previously fitted model
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            New data, where n_samples in the number of samples
            and n_features is the number of features.
        max_iter : int, optional
            Maximum number of iterations in iterated-pseudocount estimation.
        tol: double, optional
            Tolerance value used in stopping condition.
        Returns
        -------
        doc_topic : array-like, shape (n_samples, n_topics)
            Point estimate of the document-topic distributions
        Note
        ----
        This uses the "iterated pseudo-counts" approach described
        in Wallach et al. (2009) and discussed in Buntine (2009).
        """
        if isinstance(X, np.ndarray):
            # in case user passes a (non-sparse) array of shape (n_features,)
            # turn it into an array of shape (1, n_features)
            X = np.atleast_2d(X)
        doc_topic = np.empty((X.shape[0], self.n_topics))
        WS, DS = lda.utils.matrix_to_lists(X)
        # TODO: this loop is parallelizable
        for d in np.unique(DS):
            doc_topic[d] = self._transform_single(WS[DS == d], max_iter, tol)
        return doc_topic

    def _transform_single(self, doc, max_iter, tol):
        """Transform a single document according to the previously fit model
        Parameters
        ----------
        X : 1D numpy array of integers
            Each element represents a word in the document
        max_iter : int
            Maximum number of iterations in iterated-pseudocount estimation.
        tol: double
            Tolerance value used in stopping condition.
        Returns
        -------
        doc_topic : 1D numpy array of length n_topics
            Point estimate of the topic distributions for document
        Note
        ----
        See Note in `transform` documentation.
        """
        PZS = np.zeros((len(doc), self.n_topics))
        for iteration in range(max_iter + 1): # +1 is for initialization
            PZS_new = self.components_[:, doc].T
            PZS_new *= (PZS.sum(axis=0) - PZS + self.alpha)
            PZS_new /= PZS_new.sum(axis=1)[:, np.newaxis] # vector to single column matrix
            delta_naive = np.abs(PZS_new - PZS).sum()
            logger.debug('transform iter {}, delta {}'.format(iteration, delta_naive))
            PZS = PZS_new
            if delta_naive < tol:
                break
        theta_doc = PZS.sum(axis=0) / PZS.sum()
        assert len(theta_doc) == self.n_topics
        assert theta_doc.shape == (self.n_topics,)
        return theta_doc

    def _fit(self, X, seed_topics, seed_confidence):
        """Fit the model to the data X
        Parameters
        ----------
        X: array-like, shape (n_samples, n_features)
            Training vector, where n_samples in the number of samples and
            n_features is the number of features. Sparse matrix allowed.
        """
        random_state = lda.utils.check_random_state(self.random_state)
        rands = self._rands.copy()

        self._initialize(X, seed_topics, seed_confidence)
        for it in range(self.n_iter):
            # FIXME: using numpy.roll with a random shift might be faster
            random_state.shuffle(rands)
            if it % self.refresh == 0:
                ll = self.loglikelihood()
                logger.info("<{}> log likelihood: {:.0f}".format(it, ll))
                # keep track of loglikelihoods for monitoring convergence
                self.loglikelihoods_.append(ll)
            self._sample_topics(rands)
        ll = self.loglikelihood()
        logger.info("<{}> log likelihood: {:.0f}".format(self.n_iter - 1, ll))
        # note: numpy /= is integer division
        self.components_ = (self.nzw_ + self.eta).astype(float)
        self.components_ /= np.sum(self.components_, axis=1)[:, np.newaxis]
        self.topic_word_ = self.components_

        self.word_topic_ = (self.nzw_ + self.eta).astype(float)
        self.word_topic_ /= np.sum(self.word_topic_, axis=0)[np.newaxis, :]
        self.word_topic_ = self.word_topic_.T
        self.doc_topic_ = (self.ndz_ + self.alpha).astype(float)
        self.doc_topic_ /= np.sum(self.doc_topic_, axis=1)[:, np.newaxis]

        # delete attributes no longer needed after fitting to save memory and reduce clutter
        del self.WS
        del self.DS
        del self.ZS
        return self

    def _initialize(self, X, seed_topics, seed_confidence):
        """Initialize the document topic distribution.
        topic word distribution, etc.
        Parameters
        ----------
        seed_topics: type=dict, value={2:0, 256:0, 412:1, 113:1}
        """
        D, W = X.shape
        N = int(X.sum())
        n_topics = self.n_topics
        n_iter = self.n_iter
        logger.info("n_documents: {}".format(D))
        logger.info("vocab_size: {}".format(W))
        logger.info("n_words: {}".format(N))
        logger.info("n_topics: {}".format(n_topics))
        logger.info("n_iter: {}".format(n_iter))

        self.beta = 0.1
        self.nzw_ = nzw_ = np.zeros((n_topics, W), dtype=np.intc) # + self.beta
        self.ndz_ = ndz_ = np.zeros((D, n_topics), dtype=np.intc) # + self.alpha
        self.nz_ = nz_ = np.zeros(n_topics, dtype=np.intc)# + W * self.beta

        self.WS, self.DS = WS, DS = lda.utils.matrix_to_lists(X)
        self.ZS = ZS = np.empty_like(self.WS, dtype=np.intc)
        np.testing.assert_equal(N, len(WS))

        # seeded Initialization
        count_testing = 0
        for i in range(N):
            w, d = WS[i], DS[i]
            if w not in seed_topics:
                continue
            # check if seeded initialization
            if w in seed_topics and random.random() < seed_confidence:
                z_new = seed_topics[w]
            else:
                z_new = i % n_topics
            ZS[i] = z_new
            ndz_[d, z_new] += 1
            nzw_[z_new, w] += 1
            nz_[z_new] += 1

        # Non seeded Initialization
        for i in range(N):
            w, d = WS[i], DS[i]
            if w in seed_topics:
                continue
            z_new = i % n_topics
            ZS[i] = z_new
            ndz_[d, z_new] += 1
            nzw_[z_new, w] += 1
            nz_[z_new] += 1

        self.loglikelihoods_ = []

        self.nzw_ = nzw_.astype(np.intc)
        self.ndz_ = ndz_.astype(np.intc)
        self.nz_ = nz_.astype(np.intc)

    def purge_extra_matrices(self):
        """Clears out word topic. document topic. and internal matrix.
        Once this method is used. don't call fit_transform again.
        Just use the model for predictions.
        """
        del self.topic_word_
        del self.word_topic_
        del self.doc_topic_
        del self.nzw_
        del self.ndz_
        del self.nz_

    def loglikelihood(self):
        """Calculate complete log likelihood, log p(w,z)
        Formula used is log p(w,z) = log p(w|z) + log p(z)
        """
        nzw, ndz, nz = self.nzw_, self.ndz_, self.nz_
        alpha = self.alpha
        eta = self.eta
        nd = np.sum(ndz, axis=1).astype(np.intc)
        return lda._lda._loglikelihood(nzw, ndz, nz, nd, alpha, eta)

    def _sample_topics(self, rands):
        """Samples all topic assignments. Called once per iteration."""
        n_topics, vocab_size = self.nzw_.shape
        alpha = np.repeat(self.alpha, n_topics).astype(np.float64)
        eta = np.repeat(self.eta, vocab_size).astype(np.float64)
        lda._lda._sample_topics(self.WS, self.DS, self.ZS, self.nzw_, self.ndz_, self.nz_,
                                            alpha, eta, rands)

# LDA dataset preparation

In [36]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import string

In [78]:
import pandas as pd 
medical_df = pd.read_csv("/Users/sdeshpande/Desktop/bioinformatices/full_articlesLDA.csv")

In [79]:
medical_df.head()

,Unnamed: 0,paper_id,title,abstract,body_text,doi,title_abstract_body
0,0,0001418189999fea7f7cbe3e82703d71c85a6fe5,Absence of surface expression of feline infect...,Feline infectious peritonitis virus (FIPV) pos...,Feline infectious peritonitis (FIP) is a fatal...,http://doi.org/10.1016/j.vetmic.2006.11.026,Absence of surface expression of feline infect...
1,1,000affa746a03f1fe4e3b3ef1a62fdfa9b9ac52a,Correlation between antimicrobial consumption ...,Objectives: This study was conducted to invest...,The incidence of health-care-associated infect...,http://doi.org/10.1016/j.jmii.2013.10.008,Correlation between antimicrobial consumption ...
2,2,000e754142ba65ef77c6fdffcbcbe824e141ea7b,Laboratory-based surveillance of hospital-acqu...,"Of 7,772 laboratory-confirmed cases of respira...",The human respiratory viruses include adenovir...,http://doi.org/10.1016/j.ajic.2017.01.009,Laboratory-based surveillance of hospital-acqu...
3,3,000eec3f1e93c3792454ac59415c928ce3a6b4ad,Pneumonie virale sévère de l'immunocompétent V...,Reçu et accepté le 7 février 2004 Les infectio...,Les pathologies infectieuses respiratoires son...,http://doi.org/10.1016/j.reaurg.2004.02.009,Pneumonie virale sévère de l'immunocompétent V...
4,4,001259ae6d9bfa9376894f61aa6b6c5f18be2177,Microheterogeneity of S-glycoprotein of mouse ...,"IEF, isoelectric focusing; NC, nitrocellulose;...",(Accepted 10 January 1992) is a neurotropic co...,http://doi.org/10.1016/0166-0934(92)90173-B,Microheterogeneity of S-glycoprotein of mouse ...


In [80]:
medical_df = medical_df[["doi","abstract"]]
medical_df.head(10)

,doi,abstract
0,http://doi.org/10.1016/j.vetmic.2006.11.026,Feline infectious peritonitis virus (FIPV) pos...
1,http://doi.org/10.1016/j.jmii.2013.10.008,Objectives: This study was conducted to invest...
2,http://doi.org/10.1016/j.ajic.2017.01.009,"Of 7,772 laboratory-confirmed cases of respira..."
3,http://doi.org/10.1016/j.reaurg.2004.02.009,Reçu et accepté le 7 février 2004 Les infectio...
4,http://doi.org/10.1016/0166-0934(92)90173-B,"IEF, isoelectric focusing; NC, nitrocellulose;..."
5,http://doi.org/10.3389/fimmu.2015.00527,Dendritic cells (DCs) are specialized antigen-...
6,http://doi.org/10.1101/2020.01.10.901801,word count: 194 22 Text word count: 5168 23 24...
7,http://doi.org/10.1016/0166-0934(94)00119-2,The effects of centrifugation on the ability o...
8,http://doi.org/10.1016/j.econmod.2016.02.017,We review the basic concepts of intervention a...
9,http://doi.org/10.1016/j.cimid.2018.09.011,The present study evaluated the changes in sal...


In [81]:
medical_df = medical_df[0:10000]

In [54]:
# # Preprocessing_type_1
# medical_df['abstract'] = medical_df['abstract'].str.replace("[^a-zA-Z#]", " ")

# stopwords_list = stopwords.words('english')
# punctuations = list(set(string.punctuation))

# unwanted_list=punctuations+stopwords_list

# def clean_text_initial(text):
#     text = ' '.join([x.lower() for x in word_tokenize(text) if x.lower() not in unwanted_list and len(x)>1])
#     text = ' '.join([x.lower() for x in word_tokenize(text) if nltk.pos_tag([x])[0][1].startswith("NN") or nltk.pos_tag([x])[0][1].startswith("JJ")])
#     return text.strip()

# medical_df["abstract_clean"] = medical_df['abstract'].apply(lambda text:clean_text_initial(str(text)))
# medical_df.head()

,doi,abstract,abstract_clean
0,http://doi.org/10.1016/j.vetmic.2006.11.026,Feline infectious peritonitis virus FIPV pos...,feline infectious peritonitis virus fipv posit...
1,http://doi.org/10.1016/j.jmii.2013.10.008,Objectives This study was conducted to invest...,objectives study investigate correlation antib...
2,http://doi.org/10.1016/j.ajic.2017.01.009,Of laboratory confirmed cases of respira...,laboratory cases respiratory viral infection p...
3,http://doi.org/10.1016/j.reaurg.2004.02.009,Re u et accept le f vrier Les infectio...,et accept le vrier les infections virales resp...
4,http://doi.org/10.1016/0166-0934(92)90173-B,IEF isoelectric focusing NC nitrocellulose ...,ief isoelectric nc tbs tris saline ph tbs milk...


In [97]:
import re 
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

In [99]:
stopwords_list = stopwords.words('english')
punctuations = list(set(string.punctuation))

unwanted_list=punctuations+stopwords_list

In [106]:
# Preprocessing part 2 - custom tokenizer
def get_wordnet_pos(word):
    '''tags parts of speech to tokens
    Expects a string and outputs the string and 
    its part of speech'''
    
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def word_lemmatizer(text):
    '''lemamtizes the tokens based on their part of speech'''
    
    lemmatizer = WordNetLemmatizer()
    text = lemmatizer.lemmatize(text, get_wordnet_pos(text))
    return text

def reflection_tokenizer(text):
    '''expects a string an returns a list of lemmatized tokens 
    and removes the stop words. Tokens are lower cased and 
    non- alphanumeric characters as well as numbers removed. '''

    text=re.sub(r'\d+', '', text) #removes numbers
    text = text.lower()
    tokens = [word for word in word_tokenize(text)]
    tokens = [word for word in tokens if len(word) >= 3]
    #removes smaller than 3 character
    tokens = [word_lemmatizer(w) for w in tokens]
    tokens = [s for s in tokens if s not in unwanted_list]
    text = ' '.join(tokens)
    return text

In [107]:
medical_df["abstract_clean"] = medical_df['abstract'].apply(lambda text: reflection_tokenizer(str(text)))
medical_df.head()

,doi,abstract,abstract_clean
0,http://doi.org/10.1016/j.vetmic.2006.11.026,Feline infectious peritonitis virus (FIPV) pos...,feline infectious peritonitis virus fipv posit...
1,http://doi.org/10.1016/j.jmii.2013.10.008,Objectives: This study was conducted to invest...,objective study conduct investigate correlatio...
2,http://doi.org/10.1016/j.ajic.2017.01.009,"Of 7,772 laboratory-confirmed cases of respira...",laboratory-confirmed case respiratory viral in...
3,http://doi.org/10.1016/j.reaurg.2004.02.009,Reçu et accepté le 7 février 2004 Les infectio...,reçu accepté février le infection virales resp...
4,http://doi.org/10.1016/0166-0934(92)90173-B,"IEF, isoelectric focusing; NC, nitrocellulose;...",ief isoelectric focus nitrocellulose tb tris-b...


# Get guided LDA topics

In [108]:
corpus= medical_df["abstract_clean"].tolist()
vocab=list(set(word_tokenize(" ".join(medical_df["abstract_clean"]))))
vectorizer = CountVectorizer(ngram_range=(1,1),vocabulary=vocab)
X = vectorizer.fit_transform(corpus)
word2id=vectorizer.vocabulary_

In [115]:
seed_topic_list = [['expression', 'diagnostic', 'test', 'experiment'],
['mouse', 'feline' , 'mammal', 'monkey', 'canine'],
['bacterium', 'virus', 'microorganism', 'cell', 'microbial', 'pathogen'],
['laboratory', 'hospital', 'pharmaceutical'],
['protein', 'albumin', 'glycoprotein', 'antibody', 'rna']]

In [113]:
model = GuidedLDA(n_topics=5, n_iter=2000, random_state=7, refresh=20,alpha=0.01,eta=0.01)

In [116]:
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id

In [117]:
model.fit(X, seed_topics=seed_topics, seed_confidence=0.15)

INFO:lda:n_documents: 10000
INFO:lda:vocab_size: 68846
INFO:lda:n_words: 1385257
INFO:lda:n_topics: 5
INFO:lda:n_iter: 2000
INFO:lda:<0> log likelihood: -14118818
INFO:lda:<20> log likelihood: -11763792
INFO:lda:<40> log likelihood: -11562860
INFO:lda:<60> log likelihood: -11501346
INFO:lda:<80> log likelihood: -11475212
INFO:lda:<100> log likelihood: -11460581
INFO:lda:<120> log likelihood: -11454703
INFO:lda:<140> log likelihood: -11446630
INFO:lda:<160> log likelihood: -11442857
INFO:lda:<180> log likelihood: -11438055
INFO:lda:<200> log likelihood: -11434652
INFO:lda:<220> log likelihood: -11432140
INFO:lda:<240> log likelihood: -11430781
INFO:lda:<260> log likelihood: -11427761
INFO:lda:<280> log likelihood: -11427139
INFO:lda:<300> log likelihood: -11427324
INFO:lda:<320> log likelihood: -11426038
INFO:lda:<340> log likelihood: -11425803
INFO:lda:<360> log likelihood: -11424713
INFO:lda:<380> log likelihood: -11423768
INFO:lda:<400> log likelihood: -11422468
INFO:lda:<420> log li

In [118]:
n_top_words = 10
topic_word = model.topic_word_
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: virus sample use strain sequence assay pcr study detection detect
Topic 1: disease health use model study data outbreak system public method
Topic 2: cell infection response virus mouse immune expression vaccine study induced
Topic 3: patient respiratory infection virus study case child clinical influenza group
Topic 4: protein virus cell rna viral cov sars host human use


In [119]:
topic_num_name = {"Topic 0":"analysis",
                  "Topic 1":"animals",
                  "Topic 2":"microorganism",
                  "Topic 3":"place",
                  "Topic 4":"cellular component"}   

In [120]:
def get_doc_topics(model_glda,X,num_topics,dataframe,col_name):
    """
    A function which creates dataframe with documents, their dominant topic, along with their probabilities
    
    Parameters
    -------------
    model_glda - Guided LDA trained model
    X - Document term frequency table
    num_topics - Number of topics the model was trained for
    dataframe - Dataframe consisting of cleaned text column
    col_name - Column name in dataframe holding cleaned text
    
    Returns
    -------------
    A dataframe with document number, topic, probability of topic
    """
    df_doc_top = pd.DataFrame()
    final_list = []
    for index in range(len(dataframe[col_name])):
        word_id_dict = dict((x,y) for x,y in zip([x for x in range(num_topics)],np.round(model.transform(X[index])*100,1).tolist()[0]))
        word_score_list = []
        for index in range(num_topics):
            try:
                value = word_id_dict[index]
            except:
                value = 0
            word_score_list.append(value)
        final_list.append(word_score_list)

    df_doc_top = pd.DataFrame(final_list)
    df_doc_top.columns = ['Topic ' + str(i) for i in range(num_topics)]
    df_doc_top.index = ['Document ' + str(i) for i in range(len(dataframe[col_name]))]

    df_doc_top["Dominant_Topic"] = df_doc_top.idxmax(axis=1).tolist()
    df_doc_top["Topic_Probability"] = df_doc_top.max(axis=1).tolist()
    document_df = df_doc_top.reset_index().rename(columns={"index":"Document"})[["Document","Dominant_Topic","Topic_Probability"]]

    return document_df

In [121]:
document_df=get_doc_topics(model,X,5,medical_df,"abstract_clean")

a:all zero column in document-term matrix found


In [64]:
document_df.head()

,Document,Dominant_Topic,Topic_Probability
0,Document 0,Topic 4,53.3
1,Document 1,Topic 2,55.1
2,Document 2,Topic 2,99.9
3,Document 3,Topic 2,92.6
4,Document 4,Topic 0,59.0
